In [1]:
!pip install lifelines

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 349.3/349.3 kB 6.1 MB/s eta 0:00:00:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.5/52.5 kB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.5/94.5 kB 3.9 MB/s eta 0:00:00
  Created wheel for autograd-gamma: filename=autograd_gamma-0.5.0-py3-none-any.whl size=4030 sha256=623d2636293e1400dfcc685b8966ddbb12bde335fc20d2a2aa165e40d8dc9f16
  Stored in directory: /root/.cache/pip/wheels/25/cc/e0/ef2969164144c899fedb22b338f6703e2b9cf46eeebf254991
Successfully built autograd-gamma


In [2]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from lifelines import CoxPHFitter
from lifelines.utils import concordance_index

# Load data
train = pd.read_csv('/mnt/data/train.csv')
test = pd.read_csv('/mnt/data/test.csv')
data_dict = pd.read_csv('/mnt/data/data_dictionary.csv')

# Quick data inspection
print("Train Data Shape:", train.shape)
print("Test Data Shape:", test.shape)
print("Columns in Train:", train.columns)

# Extract target variables
event_col = 'efs'
time_col = 'efs_time'

# Preprocess the training data
# Encode categorical variables
categorical_cols = train.select_dtypes(include=['object', 'category']).columns
label_encoders = {}

for col in categorical_cols:
    le = LabelEncoder()
    train[col] = train[col].fillna('Missing')
    train[col] = le.fit_transform(train[col])
    label_encoders[col] = le

# Scale numerical variables
numerical_cols = train.select_dtypes(include=['float64', 'int64']).columns.drop([time_col])
scaler = StandardScaler()
train[numerical_cols] = scaler.fit_transform(train[numerical_cols])

# Stratified Concordance Index calculation
def stratified_c_index(df, risk_scores, time_col, event_col, stratify_col):
    groups = df[stratify_col].unique()
    c_indices = []

    for group in groups:
        group_data = df[df[stratify_col] == group]
        c_index = concordance_index(group_data[time_col], risk_scores[group_data.index], group_data[event_col])
        c_indices.append(c_index)

    return np.mean(c_indices) - np.std(c_indices)

# Prepare data for CoxPH model
X = train.drop(columns=[time_col, event_col, 'race_group'])
y = train[[time_col, event_col]]

cox_model = CoxPHFitter()
cox_model.fit(pd.concat([X, y], axis=1), duration_col=time_col, event_col=event_col)

# Calculate risk scores and Stratified Concordance Index
train['risk_score'] = cox_model.predict_partial_hazard(X)
c_index_score = stratified_c_index(train, train['risk_score'], time_col, event_col, stratify_col='race_group')

print("Stratified Concordance Index on Train:", c_index_score)

# Preprocess test data
for col in categorical_cols:
    test[col] = test[col].fillna('Missing')
    test[col] = label_encoders[col].transform(test[col])

test[numerical_cols] = scaler.transform(test[numerical_cols])

# Predict on test set
test['risk_score'] = cox_model.predict_partial_hazard(test)
submission = test[['ID', 'risk_score']]
submission.columns = ['ID', 'prediction']

# Save submission file
submission.to_csv('/mnt/data/submission.csv', index=False)
print("Submission file saved as 'submission.csv'")


FileNotFoundError: [Errno 2] No such file or directory: '/mnt/data/train.csv'